## Youtube API Data Collection: Political Leaders on Youtube on Channels, Videos, Engagement metrics

In [92]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
import time

### Scrape Channel statistics from youtube. Channel Data: Channel names, Descriptions, Subscribers, Views, Total Videos

View Source Page, and Find: https://www.youtube.com/channel/ (id)

**Channels IDs | Political Leaders**

'UCiFpwKZAaXxGGE-mrQ1-QMA' #KyriakosMytsotakis-New Democracy)\
'UCQRkc4_IZv27CyYw2tdAi9A' #StefanosKasselakis-Syriza)\
'UCdcyrXU9tbhpYzu2ZFL0cCA' #NikosAndroulakis-Pasok\
'UCJkuXSz3IIBir_IkHSyQ-2w' #KyriakosVelopoulos-EllinikiLysi\
'UC93M_NummpHjSqd5MngCLhA' #Spartiates-VasilisStigas\
'UCzDdvVB843U1XybJiTYVVvg' #VasilisHaritsis-NeaAristera\
'UC_wSXkcCtfqvuFcmhGWTBEg' #DimitrisNatsios-Niki\
'UC8dsz9dtUvIf7HqP5jrC_Jw' #ZoeKonstantopoulou-PleusiEleutherias\
'UCC5GG5tZf0APYlBbzup9J9A' #DimitrisKoutsoumpas-KKE\

In [93]:
api_key = 'my key'
channel_ids = [
    'UCiFpwKZAaXxGGE-mrQ1-QMA',  # KyriakosMytsotakis-New Democracy
    'UCQRkc4_IZv27CyYw2tdAi9A',  # StefanosKasselakis-Syriza
    'UCdcyrXU9tbhpYzu2ZFL0cCA',  # NikosAndroulakis-Pasok
    'UCJkuXSz3IIBir_IkHSyQ-2w',  # KyriakosVelopoulos-EllinikiLysi
    'UC93M_NummpHjSqd5MngCLhA',  # Spartiates-VasilisStigas
    'UCzDdvVB843U1XybJiTYVVvg',  # AlexandrosHaritsis-NeaAristera
    'UC_wSXkcCtfqvuFcmhGWTBEg',  # DimitrisNatsios-Niki
    'UC8dsz9dtUvIf7HqP5jrC_Jw',  # ZoeKonstantopoulou-PleusiEleutherias
    'UCC5GG5tZf0APYlBbzup9J9A',  #DimitrisKoutsoumpas-KKE
            ]

youtube= build('youtube', 'v3', developerKey=api_key)

In [94]:
#Function to get Channel Data(Channel_names,Descriptions, Subscribers, Views, Total Videos)
def get_channel_stats(youtube, channel_ids):
    all_data=[]
    request = youtube.channels().list(
        part = 'snippet,contentDetails,statistics',
            id= ','.join(channel_ids))
    
    response = request.execute()

    for i in range(len(response['items'])):
        data = dict(
                        Channel_name=response['items'][i]['snippet']['title'],
                        Description=response['items'][i]['snippet']['description'],
                        Subscribers=response['items'][i]['statistics']['subscriberCount'],
                        ViewCount=response['items'][i]['statistics']['viewCount'],
                        Total_Videos=response['items'][i]['statistics']['videoCount'],
                        Playlist_id=response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
                    )
        all_data.append(data)

    return all_data

In [95]:
channel_all_data= get_channel_stats(youtube, channel_ids)

In [96]:
channelData= pd.DataFrame(channel_all_data)

In [7]:
#ChannelData change variable types

In [97]:
print(channelData.dtypes)

Channel_name    object
Description     object
Subscribers     object
ViewCount       object
Total_Videos    object
Playlist_id     object
dtype: object


In [98]:
channelData['Subscribers'] = channelData['Subscribers'].astype('int')
channelData['ViewCount'] = channelData['ViewCount'].astype('int')
channelData['Total_Videos'] = channelData['Total_Videos'].astype('int')

In [99]:
# Replace these channel names  with the actual channel names English equivalents
channel_newnames = {'ΚΟΜΜΟΥΝΙΣΤΙΚΟ ΚΟΜΜΑ ΕΛΛΑΔΑΣ': 'Kommounistiko Komma Elladas',
                   'Kyriakos Mitsotakis': 'Kyriakos Mitsotakis',
                   'Αλέξης Χαρίτσης - Alexis Charitsis': 'Alexandros Charitsis',
                   'Stefanos Kasselakis': 'Stefanos Kasselakis',
                   'Νίκος Ανδρουλάκης': 'Nikos Androulakis',
                   'PlefsiEleftherias': 'Plefsi Eleftherias',
                   'ΕΛΛΗΝΙΚΗ ΛΥΣΗ - ΚΥΡΙΑΚΟΣ ΒΕΛΟΠΟΥΛΟΣ': 'Elliniki Lysi - Kyriakos Velopoulos',
                   '«ΣΠΑΡΤΙΑΤΕΣ» (Επίσημη Σελίδα)': 'Spartiates (Official Page)',
                   'NIKH': 'NIKI'
                   }
channelData['Channel_name'] = channelData['Channel_name'].replace(channel_newnames)

In [100]:
channelData.to_csv('channelData.csv',index=False)

In [12]:
channelData = pd.read_csv('channelData.csv')

### Scrape Video details form the youtube channels.

**Video ids list (All uploaded videos of a channel) | PAGINATION**

In [83]:
Playlist_id = channelData.loc[channelData['Channel_name']=='NIKI','Playlist_id'].iloc[0]

In [84]:
def get_videos_ids(youtube, Playlist_id):

    request = youtube.playlistItems().list(
                    part='contentDetails',
                    playlistId = Playlist_id,
                    maxResults=50)
    response = request.execute()

    video_ids= []

    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token= response.get('nextPageToken')
    more_pages = True

    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part= 'contentDetails',
                        playlistId = Playlist_id,
                        maxResults=50,
                        pageToken= next_page_token)
            response = request.execute()

            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])

                next_page_token = response.get('nextPageToken')
    
    return video_ids

**Video ids list (All uploaded videos of a channel) used to get the details of all the videos have been uploaded.**

In [85]:
video_ids = get_videos_ids(youtube, Playlist_id)

In [86]:
def get_video_details(youtube, video_ids):

    all_videos_stats = []
    
    for i in range(0, len(video_ids),50):
        request = youtube.videos().list(
                            part='snippet,statistics',
                            id= ','.join(video_ids[i:i+50]))
        
        response =request.execute()

        for video in response['items']:
            video_stats = dict(
                            Id= video['id'],
                            Title= video['snippet']['title'],
                            Published_Date= video['snippet']['publishedAt'],
                            Description= video['snippet']['description'],
                            Tags= video['snippet'].get('tags', []),
                            Views = video['statistics'].get('viewCount',0),
                            Likes = video['statistics'].get('likeCount', 0),
                            Dislikes= video['statistics'].get('dislikeCount',0),
                            Comments = video['statistics'].get('commentCount',0)
                              )
            all_videos_stats.append(video_stats)
            
    return all_videos_stats

In [87]:
video_details = get_video_details(youtube, video_ids)

#### Dimtris Koutsoumpas | KKE 0

In [18]:
DK00_videos_data = pd.DataFrame(video_details)

In [19]:
DK00_videos_data['Views'] = DK00_videos_data['Views'].astype('int')
DK00_videos_data['Likes'] = DK00_videos_data['Likes'].astype('int')
DK00_videos_data['Dislikes'] = DK00_videos_data['Dislikes'].astype('int')
DK00_videos_data['Comments'] = DK00_videos_data['Comments'].astype('int')

In [20]:
DK00_videos_data.to_csv('DimitrisKoutsoumpas_videos_data.csv',index=False)

#### Kyriakos Mitsotakis | Nea Dimokratia 1

In [26]:
KM01_videos_data = pd.DataFrame(video_details)

In [27]:
KM01_videos_data['Views'] = KM01_videos_data['Views'].astype('int')
KM01_videos_data['Likes'] = KM01_videos_data['Likes'].astype('int')
KM01_videos_data['Dislikes'] = KM01_videos_data['Dislikes'].astype('int')
KM01_videos_data['Comments'] = KM01_videos_data['Comments'].astype('int')

In [28]:
KM01_videos_data.to_csv('KyriakosMitsotakis_videos_data.csv',index=False)

#### Alexis Haritsis | Nea Aristera 2

In [14]:
AH02_videos_data = pd.DataFrame(video_details)

In [15]:
AH02_videos_data['Views'] = AH02_videos_data['Views'].astype('int')
AH02_videos_data['Likes'] = AH02_videos_data['Likes'].astype('int')
AH02_videos_data['Dislikes'] = AH02_videos_data['Dislikes'].astype('int')
AH02_videos_data['Comments'] = AH02_videos_data['Comments'].astype('int')

In [17]:
AH02_videos_data.to_csv('AlexandrosHaritsis_videos_data.csv',index=False)

#### Stefanos Kasselakis | SYRIZA 3

In [55]:
SK03_videos_data = pd.DataFrame(video_details)

In [56]:
SK03_videos_data

,Id,Title,Published_Date,Description,Tags,Views,Likes,Dislikes,Comments
0,aaasi9o2jtM,Επιστολική ψήφος: Επιθυμεί η ΝΔ τον ειλικρινή ...,2024-01-23T18:09:52Z,Γίνεται μια μεγάλη συζήτηση και εδώ στη Βουλή ...,[],3001,209,0,51
1,2BGkTa2vwgY,Ομιλία στη συνεδρίαση της ΚΕ του ΣΥΡΙΖΑ Προοδε...,2024-01-13T11:56:01Z,Ομιλία στη συνεδρίαση της ΚΕ του ΣΥΡΙΖΑ Προοδε...,[],8053,233,0,46
2,-6tUsUp8v-0,Συνέντευξη στο κεντρικό δελτίο ειδήσεων του st...,2024-01-13T10:13:57Z,-Την ώρα που ο Πρωθυπουργός φοβάται και στρέφε...,[],2513,106,0,35
3,hNsY44Jocpc,Δε θα αφήνω αναπάντητη τη γιγαντιαία προπαγάνδ...,2024-01-08T18:43:18Z,"Το έχω πει από την πρώτη στιγμή, πριν καν εκλε...",[],3579,291,0,56
4,SJI7Wh6Gj3o,Καλή Χρονιά!,2023-12-28T15:47:42Z,,[],8195,449,0,48
...,...,...,...,...,...,...,...,...,...
63,ZxDtwrpMuZY,Εσωκομματικά να πάμε πιο Αριστερά. Να αποφασίζ...,2023-08-14T15:49:37Z,,[],1549,50,0,0
64,PrLd-fx6xjE,Η Νέα Δημοκρατία κυβερνά 4 χρόνια. Δεν ανέλαβε...,2023-08-14T15:47:56Z,,[],6522,125,0,3
65,HN2lUD7kOPM,Οφείλει να υπάρξει παραίτηση του κ. Οικονόμου....,2023-08-14T15:47:11Z,,[],7776,96,0,9
66,57Z_UWL6sso,Σημερινές μου παρεμβάσεις σε τηλεοπτικούς σταθ...,2023-08-14T15:44:22Z,Αποσπάσματα από σημερινές μου παρεμβάσεις στου...,[],2961,80,0,0


In [57]:
SK03_videos_data['Views'] = SK03_videos_data['Views'].astype('int')
SK03_videos_data['Likes'] = SK03_videos_data['Likes'].astype('int')
SK03_videos_data['Dislikes'] = SK03_videos_data['Dislikes'].astype('int')
SK03_videos_data['Comments'] = SK03_videos_data['Comments'].astype('int')

In [58]:
SK03_videos_data.to_csv('StefanosKasselakis_videos_data.csv',index=False)

#### Nikos Androulakis | PASOK 4

In [46]:
NA04_videos_data = pd.DataFrame(video_details)

In [47]:
NA04_videos_data['Views'] = NA04_videos_data['Views'].astype('int')
NA04_videos_data['Likes'] = NA04_videos_data['Likes'].astype('int')
NA04_videos_data['Dislikes'] = NA04_videos_data['Dislikes'].astype('int')
NA04_videos_data['Comments'] = NA04_videos_data['Comments'].astype('int')

In [48]:
NA04_videos_data.to_csv('NikosAndroulakis_videos_data.csv',index=False)

#### Zoe Konstantopoulou | Pleusi Eleftherias 5

In [64]:
PE05_videos_data = pd.DataFrame(video_details)

In [65]:
PE05_videos_data['Views'] = PE05_videos_data['Views'].astype('int')
PE05_videos_data['Likes'] = PE05_videos_data['Likes'].astype('int')
PE05_videos_data['Dislikes'] = PE05_videos_data['Dislikes'].astype('int')
PE05_videos_data['Comments'] = PE05_videos_data['Comments'].astype('int')

In [66]:
PE05_videos_data.to_csv('ZoeKonstantopoulou_videos_data.csv',index=False)

#### Kiriakos Velopoulos | Elliniki Lysi 6

In [72]:
KV06_videos_data = pd.DataFrame(video_details)

In [73]:
KV06_videos_data['Views'] = KV06_videos_data['Views'].astype('int')
KV06_videos_data['Likes'] = KV06_videos_data['Likes'].astype('int')
KV06_videos_data['Dislikes'] = KV06_videos_data['Dislikes'].astype('int')
KV06_videos_data['Comments'] = KV06_videos_data['Comments'].astype('int')

In [74]:
KV06_videos_data.to_csv('KyriakosVelopoulos_videos_data.csv',index=False)

#### Vasilis Stigas | Spartiates 7

In [80]:
VS07_videos_data = pd.DataFrame(video_details)

In [81]:
VS07_videos_data['Views'] = VS07_videos_data['Views'].astype('int')
VS07_videos_data['Likes'] = VS07_videos_data['Likes'].astype('int')
VS07_videos_data['Dislikes'] = VS07_videos_data['Dislikes'].astype('int')
VS07_videos_data['Comments'] = VS07_videos_data['Comments'].astype('int')

In [82]:
VS07_videos_data.to_csv('VasilisStigas_videos_data.csv',index=False)

#### Dimitris Natsios | Niki 8

In [88]:
DN08_videos_data = pd.DataFrame(video_details)

In [89]:
DN08_videos_data['Views'] = DN08_videos_data['Views'].astype('int')
DN08_videos_data['Likes'] = DN08_videos_data['Likes'].astype('int')
DN08_videos_data['Dislikes'] = DN08_videos_data['Dislikes'].astype('int')
DN08_videos_data['Comments'] = DN08_videos_data['Comments'].astype('int')

In [90]:
DN08_videos_data.to_csv('DimitrisNatsios_videos_data.csv',index=False)

### Scrape Comments from videos of each youtube channel.

In [86]:
df = pd.read_csv('StefanosKasselakis_videos_data.csv')

In [87]:
df

,Id,Title,Published_Date,Description,Tags,Views,Likes,Dislikes,Comments
0,hHUbpjyLs1o,Ομιλία στη Συνεδρίαση της Κεντρικής Επιτροπής ...,2023-12-09T23:48:30Z,Ομιλία του Προέδρου του ΣΥΡΙΖΑ - Προοδευτική Σ...,[],6032,206,0,40
1,cmV6wK0geJI,Ομιλία στο 34th Annual Greek Economic Summit,2023-12-06T08:08:38Z,Ομιλία στο 34th Annual Greek Economic Summit π...,[],2767,179,0,16
2,iJp8TDnxqQE,Η ζωή είναι ένα δώρο και τη ζεις χωρίς πρόβες,2023-12-04T10:25:49Z,"Παγκόσμια ημέρα ατόμων με αναπηρία, υπάρχουν δ...",[],2442,176,0,28
3,TzkrcnjCKFk,"Οι νέοι θέλουν αξιοκρατία, ευκαιρίες και σεβασμό",2023-12-04T08:00:25Z,Ο Κυριάκος Μητσοτάκης είναι ο θείος που λέει «...,[],2708,238,0,59
4,GsWnauCfCj0,Ομιλία στη συνεδρίαση της Κ.Ο. του ΣΥΡΙΖΑ ΠΣ,2023-11-30T17:09:59Z,Ομιλία του προέδρου του ΣΥΡΙΖΑ - Προοδευτική Σ...,[],6544,274,0,54
5,dmYx0WIqrt4,"Θα γελούσα με το black χιούμορ του, αν δεν ήτα...",2023-11-22T16:45:43Z,"Σήμερα στη Βουλή, ο Κυριάκος Μητσοτάκης είπε ό...",[],4343,276,0,41
6,dndHdFBqFms,Πόσοι τολμάνε να έρθουν απροειδοποίητα εδώ;,2023-11-22T14:49:20Z,Πόσοι τολμάνε να έρθουν εδώ μέσα στον κόσμο να...,[],2677,223,0,28
7,uDOuz-WjIW0,Συγκέντρωση για τη φορολόγηση των ελεύθερων επ...,2023-11-22T14:30:08Z,NaN,[],3503,116,0,11
8,Dx9d0Ynk9No,"Συνέντευξη στο Κόκκινο 105,5 και στην Ευγενία ...",2023-11-20T15:44:23Z,Με τίμησε ο Αλέξης Τσίπρας να είμαι στο Επικρα...,[],2261,140,0,41
9,4PURZO0Cr4s,Ομιλία στο κλείσιμο των εργασιών της Κεντρικής...,2023-11-14T11:08:56Z,Δεν πρέπει κανείς να χειροκροτήσει για την απο...,[],3076,156,0,32


In [68]:
def get_video_comments(youtube, video_ids):
    all_video_comments = []

    for video_id in video_ids:
        next_page_token = None

        while True:
            try:
                request = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    textFormat='plainText',
                    maxResults=100,
                    pageToken=next_page_token
                )
    
                response = request.execute()
    
                if 'error' in response:
                    print(f"Error: {response['error']['message']}")
                    break
    
                video_comments = []
    
                for i in range(len(response['items'])):
                    comment = response['items'][i]['snippet']['topLevelComment']['snippet']
                    comment_data = dict(
                        Video_ID=video_id,
                        User_name=comment['authorDisplayName'],
                        Comment=comment['textDisplay'],
                        Comment_likes=comment['likeCount'],
                        Published_Date=comment['publishedAt']
                    )
    
                    video_comments.append(comment_data)
    
                all_video_comments.extend(video_comments)
    
                next_page_token = response.get('nextPageToken')
                if not next_page_token:
                    break
    
                time.sleep(1)
            
    return all_video_comments

In [72]:
video_ids = ['tX9YxqAQZFw','eYo1R8hXxjE','GALqtRgG9UE']

In [73]:
comments = get_video_comments(youtube, video_ids)

In [75]:
comments_data = pd.DataFrame(comments)

In [76]:
comments_data

,Video_ID,User_name,Comment,Comment_likes,Published_Date
0,tX9YxqAQZFw,@user-wl2gr6lo5o,Oi magi tou weberly charmed ant1,1,2023-12-06T10:59:59Z
1,tX9YxqAQZFw,@ihe7018,"ξύλινη γλώσσα, ξύλινη στάση του σώματος, έχει ...",1,2023-11-04T06:29:43Z
2,tX9YxqAQZFw,@user-rg9dz8xl2g,Συριζοκοπρίτες την προδοσία των Πρεσπών δεν ξε...,0,2023-10-26T10:02:03Z
3,tX9YxqAQZFw,@ivaylokirov1023,Την ίδια συνταγή πολιτικού έχουν φτιάξει οι κ...,0,2023-10-25T22:16:08Z
4,tX9YxqAQZFw,@MrStylianos1,Κασσελάκης για παλιά του άρθρα: Δεν θυμάμαι τι...,2,2023-10-22T15:23:42Z
...,...,...,...,...,...
297,GALqtRgG9UE,@user-xm2hw7yh9e,Ολλανδία θα την κάνουμε ρωτάω 2 αν ο πατέρας ...,0,2023-09-19T06:55:40Z
298,GALqtRgG9UE,@limnarxisvolvi8332,"Ο ΓΚΕΥ ΘΑ ΜΑΣ ΣΩΣΕΙ;; ΑΧ ΕΛΛΑΔΙΣΤΑΝ , ΜΕ ςΝΤΟΛ...",0,2023-09-18T09:14:13Z
299,GALqtRgG9UE,@user-wh3dc1fb2i,Μάλιστα..θα το έχω υπόψιν μου για κάποια άλλη ...,8,2023-09-13T17:44:30Z
300,GALqtRgG9UE,@user-lf5ms6yq5k,"Πες μας και την συνταγή, πώς γίνεται κάποιος ε...",27,2023-09-13T17:30:26Z
